<a href="https://colab.research.google.com/github/yuanrh2004/ML2023_HW/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS
To: /content/covid.train.csv
100% 2.49M/2.49M [00:00<00:00, 76.7MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg
To: /content/covid.test.csv
100% 993k/993k [00:00<00:00, 62.7MB/s]


In [ ]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

Some Utility Functions

In [ ]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    # 让 cuDNN 以确定性模式运行，保证相同输入时 CNN 的计算结果一致
    torch.backends.cudnn.benchmark = False
    # 关闭 cuDNN 的自动优化搜索，避免因不同输入尺寸导致计算路径不同，从而保证复现性
    np.random.seed(seed)
    # 设置 NumPy 的随机数种子，影响 numpy.random 相关的随机操作（如 np.random.rand())
    torch.manual_seed(seed)
    # 设置 PyTorch 的全局随机数种子，影响 torch.rand()、torch.randn() 等操作
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        # 设置所有 GPU 设备上的随机数种子，确保 CUDA 计算的随机性一致（如 torch.cuda.FloatTensor 的随机初始化）。

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Dataset

In [ ]:
class COVID19Dataset(Dataset):
  def __init__(self, x, y = None):  # read data and preprocess
    if y is None:
      self.y = y
    else:
      self.y = torch.FloatTensor(y)
    self.x = torch.FloatTensor(x)

  def __getitem__(self, idx):       # returns one sample at one time
    if self.y is None:
      return self.x[idx]
    else:
      return self.x[idx], self.y[idx]

  def __len__(self):                # return the size of the dataset
    return len(self.x)

# Neural Network Model


In [ ]:
class My_model(nn.Module):
  def __init__(self, input_dim):
    super(My_model, self).__init__()

    self.layers = nn.Sequential(
        nn.Linear(input_dim, config['layers'][0]),
        nn.ReLU(),
        nn.Linear(config['layers'][0], config['layers'][1]),
        nn.ReLU(),
        nn.Linear(config['layers'][1], 1)
    )

  def forward(self, x):
    x = self.layers(x)
    x = x.squeeze(1)  # 压缩为1维
    return x

# Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
def select_feat(train_data, valid_data, test_data, k):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    selector = SelectKBest(f_regression, k=k)
    result = selector.fit(raw_x_train, y_train)
    idx = np.argsort(result.scores_)[::-1]
    feat_idx = list(np.sort(idx[:k]))

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [ ]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm.
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'],
                                weight_decay=config['weight_decay'], momentum=0.9)

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device.
            pred = model(x)                     ## utilize model to get the answers
            loss = criterion(pred, y)           ## criterion is used to evaluate the loss
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Configuration

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = {
    'seed': 2252241026,
    'select_all': False,
    'k': 16,
    'weight_decay': 1e-6,
    'valid_ratio' : 0.2,
    'n_epochs': 3000,
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 300,
    'layers':[16, 8],
    'save_path': './models/model.ckpt'
}

# DataLoader

In [ ]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days)
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['k'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2160, 118)
valid_data size: (539, 118)
test_data size: (1078, 117)
number of features: 16


In [ ]:
df = pd.read_csv('./covid.train.csv')

# 打印所有列名及其索引
for idx, col in enumerate(df.columns):
    print(f"Index: {idx}, Column: {col}")

Index: 0, Column: id
Index: 1, Column: AL
Index: 2, Column: AK
Index: 3, Column: AZ
Index: 4, Column: AR
Index: 5, Column: CA
Index: 6, Column: CO
Index: 7, Column: CT
Index: 8, Column: FL
Index: 9, Column: GA
Index: 10, Column: ID
Index: 11, Column: IL
Index: 12, Column: IN
Index: 13, Column: IA
Index: 14, Column: KS
Index: 15, Column: KY
Index: 16, Column: LA
Index: 17, Column: MD
Index: 18, Column: MA
Index: 19, Column: MI
Index: 20, Column: MN
Index: 21, Column: MS
Index: 22, Column: MO
Index: 23, Column: NE
Index: 24, Column: NV
Index: 25, Column: NJ
Index: 26, Column: NM
Index: 27, Column: NY
Index: 28, Column: NC
Index: 29, Column: OH
Index: 30, Column: OK
Index: 31, Column: OR
Index: 32, Column: RI
Index: 33, Column: SC
Index: 34, Column: TX
Index: 35, Column: UT
Index: 36, Column: VA
Index: 37, Column: WA
Index: 38, Column: cli
Index: 39, Column: ili
Index: 40, Column: hh_cmnty_cli
Index: 41, Column: nohh_cmnty_cli
Index: 42, Column: wearing_mask
Index: 43, Column: travel_outs

# Start

In [ ]:
model = My_model(input_dim=x_train.shape[1]).to(device)
# put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/3000]: 100%|██████████| 9/9 [00:00<00:00, 76.33it/s, loss=81.8]


Epoch [1/3000]: Train loss: 118.8592, Valid loss: 86.6360
Saving model with loss 86.636...


Epoch [2/3000]: 100%|██████████| 9/9 [00:00<00:00, 79.82it/s, loss=34.8]


Epoch [2/3000]: Train loss: 62.2454, Valid loss: 22.6288
Saving model with loss 22.629...


Epoch [3/3000]: 100%|██████████| 9/9 [00:00<00:00, 86.33it/s, loss=6.74]


...

Epoch [1560/3000]: 100%|██████████| 9/9 [00:00<00:00, 65.00it/s, loss=1.34]


Epoch [1560/3000]: Train loss: 1.1307, Valid loss: 1.3440


Epoch [1561/3000]: 100%|██████████| 9/9 [00:00<00:00, 59.38it/s, loss=1.35]


Epoch [1561/3000]: Train loss: 1.1285, Valid loss: 1.3698


Epoch [1562/3000]:  78%|███████▊  | 7/9 [00:00<00:00, 68.78it/s, loss=0.678]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')